In [1]:
import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

sys.path.append(os.path.join(sys.path[0], '../src'))
import data
import utils
import model
from model import EncoderRNN
from model import DecoderRNN

%load_ext autoreload
%autoreload 2

## Load and process training data

In [2]:
# Change to directory containing English to Tamil transliteration data.
DATA_DIR = "../data/aksharantar_sampled/tam/"
os.chdir(DATA_DIR)
os.listdir()

['dataset.pkl', 'tam_test.csv', 'tam_train.csv', 'tam_valid.csv']

The class WordDataset reads and stores words in onehot representation.

In [3]:
# dataset = data.WordDataset() # By default reads 'tam_train.csv'.
# with open('dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [4]:
with open('dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

The above object `dataset[i]` returns i-th data point: a word-pair in one-hot represenation. 

In [5]:
x, y = dataset[0]
print(x.shape)
print(y.shape)

torch.Size([50, 128])
torch.Size([50, 128])


In [6]:
dataset.decode_eng_word(x)

'thottacharya'

In [7]:
dataset.decode_tam_word(y)

'தொட்டாச்சார்ய'

## Test Encoder

In [8]:
from model import EncoderRNN
encoder = EncoderRNN(cell_type='gru')
output, hidden = encoder.forward(x)

In [9]:
print(output.shape)
print(hidden.shape)

torch.Size([50, 16])
torch.Size([1, 16])


### Decoder

In [10]:
import model
from model import DecoderRNN

decoder = DecoderRNN(cell_type='gru')
out, hidden = decoder.forward(y[0], hidden)

torch.Size([1, 128])
torch.Size([1, 16])


/mnt/d/courses/cs6910_dl/assignment_3/notebooks/../src/model.py:80: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.out_act(output)


In [11]:
print(out.shape)
print(hidden.shape)

torch.Size([1, 128])
torch.Size([1, 16])


## Loss

In [12]:
target = torch.zeros(128)
target[0] = 1
target = torch.tensor([torch.where(target == 1)[0]])
y_hat = out
criterion = nn.NLLLoss()
criterion(y_hat, target)

tensor(4.9968, grad_fn=<NllLossBackward0>)

## Test Training

In [13]:
# def train(n_iters = 1):
#     encoder = EncoderRNN()
#     decoder = DecoderRNN()
#     for  i in range(n_iters):
